## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [5]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [17]:
classifier = Sequential()

inputShape = (32, 32, 3)

#卷積組合
classifier.add(Convolution2D(32, (3, 3), padding = 'same', activation = 'sigmoid', input_shape = inputShape))
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(epsilon = 1e-06, momentum = 0.9))
classifier.add(MaxPooling2D(pool_size=(2,2)))

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(16, (3, 3), padding = 'same', activation = 'sigmoid'))
classifier.add(BatchNormalization(epsilon = 1e-06, momentum = 0.9))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(64, activation = 'relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10, activation='softmax'))

classifier.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 16, 16)        4624      
_________________________________________________________________
batch_normalization_13 (Batc (None, 16, 16, 16)        64        
_________________________________________________________________
flatten_6 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)               

In [18]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size = 100,epochs = 12)

Epoch 1/12
500/500 [==============================] - 106s 212ms/step - loss: 1.4112 - accuracy: 0.5083
Epoch 2/12
500/500 [==============================] - 106s 212ms/step - loss: 1.0001 - accuracy: 0.6482
Epoch 3/12
500/500 [==============================] - 104s 209ms/step - loss: 0.8354 - accuracy: 0.7047
Epoch 4/12
500/500 [==============================] - 105s 210ms/step - loss: 0.7148 - accuracy: 0.7478
Epoch 5/12
500/500 [==============================] - 107s 214ms/step - loss: 0.6110 - accuracy: 0.7836
Epoch 6/12
500/500 [==============================] - 106s 212ms/step - loss: 0.5180 - accuracy: 0.8163
Epoch 7/12
500/500 [==============================] - 105s 210ms/step - loss: 0.4386 - accuracy: 0.8454
Epoch 8/12
500/500 [==============================] - 105s 211ms/step - loss: 0.3580 - accuracy: 0.8746
Epoch 9/12
500/500 [==============================] - 105s 211ms/step - loss: 0.2923 - accuracy: 0.8973
Epoch 10/12
500/500 [==============================] - 109s 218m

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [19]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.42251635e-02, 1.35273203e-05, 6.56635780e-03, 1.33564444e-02,
        5.27481854e-01, 2.06743809e-03, 1.92577529e-04, 4.08243805e-01,
        2.76790876e-02, 1.73737222e-04]], dtype=float32)